# Units and Particles

This notebook introduces some of the most commonly used functionality when working with PlasmaPy...

## Working with units

The `astropy.units` subpackage is...

In [ ]:
import astropy.units as u

## Working with particles

The `plasmapy.particles` subpackage is...

In [ ]:
from plasmapy import particles